   # Procesing Twitter Dataset and loading into MongoDB database system

### Following constrains are used to reduce the querrying time


1) Irrelevant  data fields are dropped.
 

2) Tweets are stored before retweets.


3) Order of appearance of attributes is changed.

### Importing reuired libraried

In [33]:
import os
import json
import tweepy 
import pandas as pd 
import pymongo
import time

###                                       Setting up mongoDB client connection 

In [34]:
client=pymongo.MongoClient("mongodb://localhost:27017")

Creating an empty database

In [37]:
db=client["corona3"]

In [39]:
start_time= time.time()
with open('corona-out-3.json', 'r') as f:
    print("Inserting tweets in MongoDB")
    
    retweeted_status_keys=["created_at","id_str","text","in_reply_to_status_id_str","in_reply_to_user_id_str","in_reply_to_screen_name","user","quoted_status_id_str","quote_count","reply_count","retweet_count","favorite_count","entities","lang"]
    # Process tweets
    for line in f:
        if(line=='\n'):
            continue            
        tweet = json.loads(line)
        
        dictz = {"user":None,'text':None,"entities":None,'created_at': None, 'id_str':None, "in_reply_to_screen_name":None ,"in_reply_to_user_id_str": None,"in_reply_to_status_id_str": None,"quoted_status_id_str": None, "quote_count" : None,"reply_count" : None,"retweet_count": None,"favorite_count": None, "lang": None}
        
        # Check if tweet has not been retweeted
        if "retweeted_status" not in tweet:
            # Extract information from tweet
            if "user" in tweet:
                user=tweet["user"]
                dictz["user"]={"id_str":user["id_str"]} 
            if "entities" in tweet:                             
                entities=tweet["entities"]
                dictz["entities"]={"hasgtages":entities["hashtags"]}           
            for line2 in tweet:
                if line2 in dictz:
                    dictz[line2]=tweet[line2]
                    
            # Insert tweet into MongoDB collection
            db.tweets.insert_one(dictz)
    print("Tweets Inserted")
    # Reset file pointer to beginning of file
    f.seek(0)
    
    print("Inserting Retweets")
    
    # Process retweets
    for line in f:
        if(line=='\n'):
            continue            
        tweet = json.loads(line)
        
       
        dictz = {"user":None,'text':None,"entities":None,'created_at': None, 'id_str':None,"retweeted_status": None , "in_reply_to_screen_name":None ,"in_reply_to_user_id_str": None,"in_reply_to_status_id_str": None,"quoted_status_id_str": None, "quote_count" : None,"reply_count" : None,"retweet_count": None,"favorite_count": None, "lang": None}
        
        # Check if tweet has been retweeted
        if "retweeted_status" in tweet:
            # Extract information from retweeted status
            rts= tweet["retweeted_status"]                       
            retweeted_status={}                 
            for key in retweeted_status_keys:
                if key in rts:                       
                    retweeted_status[key] = rts[key]                   
                dictz['retweeted_status'] = retweeted_status
            
            # Insert retweet into MongoDB collection
            db.tweets.insert_one(dictz)
    print("Retweets inserted")
    print("Complete")
    time_taken=time.time()-start_time
    print(" time_taken=", time_taken,"seconds")

Inserting tweets in MongoDB
Tweets Inserted
Inserting Retweets
Retweets inserted
Complete
 time_taken= 113.21362590789795 seconds
